In [1]:
#########################
### Import Code Stuff ###
#########################

### Numpy and Scipy
import numpy as np
from scipy.stats import binned_statistic, binned_statistic_2d

### Astropy FITS/Table handling
from astropy.io import fits, ascii
from astropy.table import Table, Column, vstack

#astropy coorindates/units
from astropy.coordinates import SkyCoord
import astropy.constants as const
import astropy.units as u

### Matplotlib
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams.update({'font.size': 22})

### Geometry
import lmcgeometry as lgeo

### Other
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm_notebook

In [2]:
def add_noise(quant,quant_err,distribution='normal'):
    '''
    Add noise to data and return new values
    
    Parameters:
    ----------
        quant: 1d array-like data to add noise to
        quant_err: 1d array-like object of errors for quant
        distribution: which distribution to use 'normal', 'poisson', 'uniform'
    
    return: 
    ------
        1d array-like object of data with added noise
    
    '''
    if distribution == 'normal':
        return np.random.normal(quant,quant_err)
    if distribution == 'poisson':
        return quant + np.random.poisson(quant_err)
    if distribution == 'uniform':
        return np.random.uniform(-quant_err+quant,quant+quant_err)
    
def mad(x):
    '''
    Calculate median absolute deviation:
        
        MAD = median(x_i-median(x))
        
    Inputs:
    ------
        x: data to calculate mad of 
    
    Outputs:
    -------
        mad: median absolute deviation
    '''
    return np.median(x-np.median(x))

def salaris(mh,am):
    '''
    Calculate the Salaris correction (Salaris et al. 1993)
    
    Inputs:
    ------
        mh: [M/H] 
        am: [ALPHA/H]
    
    Outputs:
    -------
        feh adjusted metallicity
    '''
    feh = mh + np.log10((0.638*(10**(am)))+0.362)
    return feh

def LMCdisk_cart(ra, dec):
    
    '''
    Calculate the position of stars in the LMC disk plane with 
    center at the LMC center in cartesian coordinates (x, y).
    This also calculates the distance to the individual stars.
    
    This follows van der Marel and Cioni 2001 
    
    Input
    - ra: right ascension of stars
    - dec: declination of stars
    
    Output
    - x_m: x coordinate
    - y_m: y coordinate
    - dis: distance to LMC star
    '''
    alph0 = np.radians(82.25) #right ascension of center of LMC
    delt0 = np.radians(-69.50) #declination of center of LMC
    pa = np.radians(149.23+90.00) #146.37 #position angle of line of nodes
    io = np.radians(25.86) #27.81 #inclination of LMC disk
    d0 = 49.90 #distance to center of LMC
    
    #convert to radians
    ra = np.radians(ra)
    dec = np.radians(dec)
    sd = np.sin(delt0)
    cd = np.cos(delt0)
    
    cr = cd*np.cos(dec)*np.cos(ra-alph0)+sd*np.sin(dec)
    srcp = -np.cos(dec)*np.sin(ra-alph0)
    srsp = cd*np.sin(dec) - sd*np.cos(dec)*np.cos(ra-alph0)
    dis = d0*np.cos(io)/(np.cos(io)*cr - np.sin(io)*np.cos(pa)*srsp + np.sin(io)*np.sin(pa)*srcp)
    
    x_m = dis*srcp
    y_m = dis*(np.cos(io)*srsp + np.sin(io)*cr) - d0*np.sin(io)
    
    return x_m, y_m, dis

#calculate absolute mag
def absmag(magnitude,distance):
    '''
    - magnitude: apparent magnitude of star
    - distance: distance to star in kpc
    Calculate the absolute magnitude of star
    '''
    absm = []
    absm.append(magnitude-5.0*np.log10(distance*1000)+5.0)
    absm = np.squeeze(np.array(absm))
    return absm

In [3]:
### Import Data
lmc = fits.getdata('/Users/joshuapovick/Desktop/Research/fits/lmc_rgbmembers.r13-l33-58672.fits.gz',1)
cln = np.where((lmc['FE_H']>-9999.0)&(lmc['AK_TARG']>-9999.0)&(lmc['LOGG']>0.0)&(lmc['M_H_ERR']>-90.0)&
                (lmc['C_FE']>-9999.0)&(lmc['N_FE']>-9999.0))

lmc = lmc[cln]

elems = ['M_H','C_FE','N_FE','O_FE','NA_FE','MG_FE','AL_FE','SI_FE','P_FE','S_FE','K_FE','CA_FE','TI_FE',
         'TIII_FE','V_FE','CR_FE','MN_FE','FE_H','CO_FE','NI_FE','CU_FE','GE_FE','RB_FE','CE_FE','ND_FE','YB_FE']

labs = ['[M_H]','[C/Fe]','[N/Fe]','[O/Fe]','[Na/Fe]','[Mg/Fe]','[Al/Fe]','[Si/Fe]','[P/Fe]','[S/Fe]','[K/Fe]',
        '[Ca/Fe]','[Ti/Fe]','[TiII/Fe]','[V/Fe]','[Cr/Fe]','[Mn/Fe]','[Fe/H]','[Co/Fe]','[Ni/Fe]','[Cu/Fe]',
        '[Ge/Fe]','[Rb/Fe]','[Ce/Fe]','[Nd/Fe]','[Yb/Fe]']

### LMC Geometry
x,y,dist = lgeo.LMCdisk_cart(lmc['RA'],lmc['DEC'])
radius = lgeo.elliptical_radius(x,y)

### PARSEC
parsec = fits.getdata('/Users/joshuapovick/Desktop/Research/parsec/parsec.fits',0)
rgb = np.where(parsec['label']==3)
parsec = parsec[rgb]

In [4]:
### Age of Universe
H0 = 74.03*(u.km/u.s)/u.Mpc
hertz = H0.to(u.km/u.s/u.pc).to(u.km/u.s/u.km)
tage = (1/hertz).to(u.yr)
ageU = tage.value

### Get Solar Fractions
# {'C':8.39,'N':7.78,'O':8.66,'Mg':7.53,'Ca':6.31,'S':7.14,'Si':7.51,'Fe':7.45}

# ac = [8.39,7.78,8.66,7.53,6.31,7.14,7.51,7.45]

# xh_sol = []
# for i in range(len(ac)):
#     xh_sol.append(ac[i]-12.0)
    
# xm_sol = 10**np.asarray(xh_sol)/sum(10**np.asarray(xh_sol))

In [36]:
### Inital Interpolator setup
# from scipy.interpolate import Rbf

val_tup = np.array([parsec['logTe'],parsec['Ksmag'],np.log10(parsec['Z']/0.02),parsec['logg']])
apo_tup = np.array([np.log10(lmc['TEFF']),lmc['K'],salaris(lmc['M_H'],lmc['ALPHA_M']),lmc['LOGG']])
# interpol = Rbf(val_arr[0],val_arr[1],val_arr[2],val_arr[3],parsec['logAge'],function='gaussian',smooth=5.0)

from scipy.interpolate import griddata,LinearNDInterpolator,RegularGridInterpolator

# age = []
# for i in apo_tup
# interpn(val_tup,parsec['logAge'],)

ages = RegularGridInterpolator(val_tup,parsec['logAge'],method='linear')

ValueError: There are 4 point arrays, but values has 1 dimensions

In [45]:
parsec.columns

ColDefs(
    name = 'Zini'; format = 'D'
    name = 'MH'; format = 'D'
    name = 'logAge'; format = 'D'
    name = 'Mini'; format = 'D'
    name = 'int_IMF'; format = 'D'
    name = 'Mass'; format = 'D'
    name = 'logL'; format = 'D'
    name = 'logTe'; format = 'D'
    name = 'logg'; format = 'D'
    name = 'label'; format = 'K'
    name = 'McoreTP'; format = 'D'
    name = 'C_O'; format = 'D'
    name = 'period0'; format = 'D'
    name = 'period1'; format = 'D'
    name = 'period2'; format = 'D'
    name = 'period3'; format = 'D'
    name = 'period4'; format = 'D'
    name = 'pmode'; format = 'K'
    name = 'Mloss'; format = 'D'
    name = 'tau1m'; format = 'D'
    name = 'X'; format = 'D'
    name = 'Y'; format = 'D'
    name = 'Xc'; format = 'D'
    name = 'Xn'; format = 'D'
    name = 'Xo'; format = 'D'
    name = 'Cexcess'; format = 'D'
    name = 'Z'; format = 'D'
    name = 'mbolmag'; format = 'D'
    name = 'Jmag'; format = 'D'
    name = 'Hmag'; format = 'D'
    name = 'Ksm